In [1]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient

st = SpaceTrackClient(identity="coolboy.seif@gmail.com", password="amr13nerA!amina!")

# Newest elset for each active Starlink sat, as TLE text (streamed)
lines = st.gp(
    iter_lines=True,                 # stream line-by-line
    object_name=op.startswith("STARLINK"),
    decay_date=None,                 # only non-decayed
    orderby="norad_cat_id",
    format="tle",
)
with open("starlink.tle", "w") as f:
    for line in lines:
        f.write(line + "\n")

In [28]:
from skyfield.api import load, EarthSatellite, wgs84
from datetime import datetime, timezone, timedelta

ts = load.timescale()
sats = load.tle_file("starlink.tle")      # many satellites at once

# time grid (every 5 min for the next hour)
t0 = datetime.now(timezone.utc)
times = ts.utc([t0 + timedelta(minutes=5*i) for i in range(30)])

# Example: compute subpoint (lat/lon/alt) for each sat at first epoch
rows = []
for sat in sats:
    geoc = sat.at(times)
    sp = geoc.subpoint()
    rows.append({
        "name": sat.name,
        "lat_deg": float(sp.latitude.degrees[0]),
        "lon_deg": float(sp.longitude.degrees[0]),
        "alt_km": float(sp.elevation.km[0]),
    })
print(f"Computed {len(rows)} satellites")



Computed 8954 satellites


In [29]:
import gnss_lib_py as glp
sp3_path = "data/COD0MGXFIN_20211180000_01D_05M_ORB.SP3"
sp3 = glp.Sp3(sp3_path)
sp3_gnss = sp3.where("gnss_id","gps")

In [36]:
# Smart LEO CSV generator aligned to SP3 epochs, with robust pre-filtering and merge-safe dtypes

# --- config ---
TLE_PATH    = "starlink.tle"
N_SATS      = 3000
SELECT_MODE = "first"          # "first" | "random" | "norad-range" | "name-prefix"
SEED        = 42
NORAD_MIN, NORAD_MAX = 44700, 99999   # widen if needed for more candidates
NAME_PREFIX = "STARLINK-"

OUTDIR      = "outputs"
OUT_CSV     = "starlink_navdata_leo_aligned_{}.csv".format(N_SATS)

GNSS_ID_TAG = "LEO"            # goes into 'gnss_id'
SV_PREFIX   = "STARLINK-"      # used in 'gnss_sv_id'

# GPS-UTC offset for your SP3 epochs (18 s since 2017-01-01)
GPS_UTC_OFFSET_SEC = 18.0

# sanity bounds for ECEF radius [km] to accept a LEO (Earth center distance)
R_E_KM = 6371.0
MIN_ALT_KM = 160.0
MAX_ALT_KM = 2000.0

RADIUS_BOUNDS_KM = (R_E_KM + MIN_ALT_KM, R_E_KM + MAX_ALT_KM)
# --- deps ---
import numpy as np, pandas as pd, pathlib, random
from datetime import datetime, timedelta, timezone
from skyfield.api import load
import gnss_lib_py as glp

# --- helpers ---
GPS_EPOCH = datetime(1980, 1, 6, tzinfo=timezone.utc)

def gps_millis_to_ts(ts, gps_millis, gps_utc_offset_sec=GPS_UTC_OFFSET_SEC):
    gps_sec = np.asarray(gps_millis, dtype=np.float64) / 1000.0
    utc_datetimes = [GPS_EPOCH + timedelta(seconds=float(s - gps_utc_offset_sec)) for s in gps_sec]
    return ts.utc(utc_datetimes)

def sat_norad(s):  # EarthSatellite -> NORAD
    return int(s.model.satnum)

def select_candidates(sats_all, mode, seed=None, nr_min=None, nr_max=None, name_prefix=None):
    """Return an ordered list of candidate satellites based on selection mode (no count yet)."""
    seq = list(sats_all)
    if mode == "first":
        return seq
    if mode == "random":
        if seed is not None:
            random.seed(seed)
        random.shuffle(seq)
        return seq
    if mode == "norad-range":
        return [s for s in seq if nr_min <= sat_norad(s) < nr_max]
    if mode == "name-prefix":
        return [s for s in seq if s.name and s.name.startswith(name_prefix)]
    raise ValueError("Bad SELECT_MODE")

def prefilter_good_leo(candidates, ts, radius_bounds_km=RADIUS_BOUNDS_KM, need=N_SATS):
    """Probe one test epoch to reject unstable/decayed sats; keep taking until we have 'need' good ones."""
    lo_km, hi_km = radius_bounds_km
    # pick a stable near-now test time; using first SP3 epoch instead (more robust to TLE age)
    t_test = ts_test

    good = []
    for sat in candidates:
        if len(good) >= need:
            break
        try:
            g = sat.at(t_test)
            sub = g.subpoint()
            lat_deg = sub.latitude.degrees
            lon_deg = sub.longitude.degrees
            alt_m   = sub.elevation.km * 1000.0
            llh = np.column_stack([lat_deg, lon_deg, alt_m])
            ecef = glp.geodetic_to_ecef(llh)
            X, Y, Z = ecef[:,0], ecef[:,1], ecef[:,2]
            r_km = float(np.linalg.norm([X[0], Y[0], Z[0]]) / 1000.0)
            if lo_km <= r_km <= hi_km:
                good.append(sat)
        except Exception:
            # any SGP4 or transform failure => skip
            continue
    return good

# --- 1) pull exact SP3 epochs already in memory ---
u_t = np.unique(np.asarray(sp3_gnss["gps_millis"]).ravel())
print(f"Aligning to {u_t.size} SP3 epochs")

# build Skyfield time scale and a *test* epoch for prefiltering (first SP3 epoch)
ts = load.timescale()
ts_test = gps_millis_to_ts(ts, [u_t[0]])  # 1-epoch Time for prefiltering

# --- 2) load TLEs, choose a robust set of good LEOs ---
sats_all = load.tle_file(TLE_PATH)
candidates = select_candidates(sats_all, SELECT_MODE, SEED, NORAD_MIN, NORAD_MAX, NAME_PREFIX)

good = prefilter_good_leo(candidates, ts, RADIUS_BOUNDS_KM, need=N_SATS)

# If not enough, broaden search automatically by sweeping the full list
if len(good) < N_SATS:
    extra_pool = [s for s in sats_all if s not in good]
    more = prefilter_good_leo(extra_pool, ts, RADIUS_BOUNDS_KM, need=N_SATS - len(good))
    good.extend(more)

if len(good) < N_SATS:
    raise RuntimeError(f"Only found {len(good)} valid LEO satellites; try widening NORAD range or using SELECT_MODE='random'.")

sel = good[:N_SATS]
print(f"Selected {len(sel)} good LEO satellites.")

# --- 3) build Skyfield Time for *all* SP3 epochs & propagate ---
times = gps_millis_to_ts(ts, u_t)  # vector of epochs

blocks = []
for sat in sel:
    g = sat.at(times)
    sub = g.subpoint()
    lat_deg = sub.latitude.degrees
    lon_deg = sub.longitude.degrees
    alt_m   = sub.elevation.km * 1000.0

    # LLH -> ECEF (meters) using gnss_lib_py (vectorized)
    llh = np.column_stack([lat_deg, lon_deg, alt_m])
    ecef = glp.geodetic_to_ecef(llh)
    X_m, Y_m, Z_m = ecef[:,0], ecef[:,1], ecef[:,2]

    norad = sat_norad(sat)
    # IDs as strings to avoid dtype/le type conflicts in NavData.copy/where paths
    sv_id_str = str(norad)
    gnss_sv_id_str = f"{SV_PREFIX}{norad}"

    # Build one DataFrame block per satellite, then concat once (fast & simple)
    block = pd.DataFrame({
        "gps_millis": u_t.astype("float64"),
        "gnss_sv_id": pd.Series([gnss_sv_id_str] * len(u_t), dtype="string"),
        "gnss_id":    pd.Series([GNSS_ID_TAG] * len(u_t),    dtype="string"),
        "sv_id":      pd.Series([sv_id_str] * len(u_t),      dtype="string"),
        "x_sv_m":     X_m.astype("float64"),
        "y_sv_m":     Y_m.astype("float64"),
        "z_sv_m":     Z_m.astype("float64"),
        "el_sv_deg":  pd.Series(np.nan, index=range(len(u_t)), dtype="float64"),
        "az_sv_deg":  pd.Series(np.nan, index=range(len(u_t)), dtype="float64"),
    })
    blocks.append(block)

df = pd.concat(blocks, ignore_index=True)

# --- 4) stable sort and save (already typed correctly) ---
df.sort_values(["gps_millis", "sv_id"], inplace=True, kind="stable")

outdir = pathlib.Path(OUTDIR); outdir.mkdir(exist_ok=True)
csv_path = outdir / OUT_CSV
df.to_csv(csv_path, index=False)

print("wrote:", csv_path, "| rows:", len(df), "| epochs:", len(u_t), "| sats:", len(sel))

# --- 5) sanity check: LEO radius should be ~6,500–8,000 km from Earth's center ---
r = np.sqrt(df["x_sv_m"]**2 + df["y_sv_m"]**2 + df["z_sv_m"]**2)
print("LEO ECEF radius [km]: min=", r.min()/1000, " median=", np.median(r)/1000, " max=", r.max()/1000)

display(df.head(8))

Aligning to 73 SP3 epochs
Selected 3000 good LEO satellites.
wrote: outputs/starlink_navdata_leo_aligned_3000.csv | rows: 219000 | epochs: 73 | sats: 3000
LEO ECEF radius [km]: min= 6524.828703266158  median= 6938.979236225748  max= 7124.131705392847


,gps_millis,gnss_sv_id,gnss_id,sv_id,x_sv_m,y_sv_m,z_sv_m,el_sv_deg,az_sv_deg
0,1.303668e+12,STARLINK-44714,LEO,44714,-1.756577e+06,-3.821331e+06,5.451164e+06,NaN,NaN
73,1.303668e+12,STARLINK-44718,LEO,44718,-2.900622e+06,-6.313666e+06,-1.560943e+05,NaN,NaN
146,1.303668e+12,STARLINK-44725,LEO,44725,3.649937e+06,2.438481e+06,-5.385425e+06,NaN,NaN
219,1.303668e+12,STARLINK-44741,LEO,44741,5.299226e+06,-1.759734e+06,-4.189200e+06,NaN,NaN
292,1.303668e+12,STARLINK-44751,LEO,44751,-3.426231e+06,-3.066182e+06,5.202589e+06,NaN,NaN
365,1.303668e+12,STARLINK-44752,LEO,44752,4.301780e+06,4.869301e+05,-5.323970e+06,NaN,NaN
438,1.303668e+12,STARLINK-44753,LEO,44753,4.093234e+06,-1.776611e+06,-5.317530e+06,NaN,NaN
511,1.303668e+12,STARLINK-44768,LEO,44768,5.607861e+05,4.260208e+06,-5.438096e+06,NaN,NaN


In [33]:
# --- sanity: inspect the unique epochs in SP3 file ---
print("Unique SP3 epochs (gps_millis):")
print(u_t[:10], "...")  # first few

# --- check epoch increments ---
dt = np.diff(u_t)
print("\nEpoch increments (ms):", np.unique(dt))
print("Epoch increments (seconds):", np.unique(dt / 1000.0))


# --- pick two specific STARLINK satellites from df ---
sv1_df = df[df["gnss_sv_id"] == "STARLINK-44753"]
sv2_df = df[df["gnss_sv_id"] == "STARLINK-44751"]

# Extract their time arrays
sv1_t = sv1_df["gps_millis"].to_numpy()
sv2_t = sv2_df["gps_millis"].to_numpy()

print("\nSTARLINK-44753 timestamps (first 10):")
print(sv1_t[:10].tolist())

print("\nSTARLINK-44751 timestamps (first 10):")
print(sv2_t[:10].tolist())

# --- sanity: check they match exactly ---
same = np.array_equal(sv1_t, sv2_t)
print("\nDo STARLINK-44753 and STARLINK-44751 have the SAME timestamps?", same)

# --- check that STARLINK-44753 timestamps match u_t exactly ---
match1 = np.array_equal(u_t.astype(float), sv1_t)
print("Does STARLINK-44753 use EXACTLY the same timestamps as u_t?", match1)


Unique SP3 epochs (gps_millis):
[1.3036680e+12 1.3036683e+12 1.3036686e+12 1.3036689e+12 1.3036692e+12
 1.3036695e+12 1.3036698e+12 1.3036701e+12 1.3036704e+12 1.3036707e+12] ...

Epoch increments (ms): [300000.]
Epoch increments (seconds): [300.]

STARLINK-44753 timestamps (first 10):
[1303668000000.0, 1303668300000.0, 1303668600000.0, 1303668900000.0, 1303669200000.0, 1303669500000.0, 1303669800000.0, 1303670100000.0, 1303670400000.0, 1303670700000.0]

STARLINK-44751 timestamps (first 10):
[1303668000000.0, 1303668300000.0, 1303668600000.0, 1303668900000.0, 1303669200000.0, 1303669500000.0, 1303669800000.0, 1303670100000.0, 1303670400000.0, 1303670700000.0]

Do STARLINK-44753 and STARLINK-44751 have the SAME timestamps? True
Does STARLINK-44753 use EXACTLY the same timestamps as u_t? True
